In [1]:
#Load in Prerequisite
import sys
! {sys.executable} -m pip install --upgrade 'xdmod-data>=1.0.0,<2.0.0' python-dotenv tabulate
#Configure formatting
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler
from IPython.display import display, Markdown
def display_df_md_table(df):
    return display(Markdown(df.replace('\n', '<br/>', regex=True).to_markdown()))
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xdmod_data.themes
pio.templates.default = "timeseries"

In [3]:
#Set up enviroment and connection
from pathlib import Path
from os.path import expanduser
xdmod_data_env_path = Path(expanduser('~/xdmod-data.env'))
try:
    with open(xdmod_data_env_path):
        pass
except FileNotFoundError:
    with open(xdmod_data_env_path, 'w') as xdmod_data_env_file:
        xdmod_data_env_file.write('XDMOD_API_TOKEN=')
    xdmod_data_env_path.chmod(0o600)
#Check if API key file is present
from dotenv import load_dotenv
load_dotenv(xdmod_data_env_path, override=True)
#Connect to sever 
#!NOTE: Update from test sever later on to generate with latest data and such
from xdmod_data.warehouse import DataWarehouse
dw = DataWarehouse('https://xdmod-dev.ccr.xdmod.org/')

#Load constant values/data into memory to be accessed later - allows for more caches hits as oppesed to calling data from server
durations = dw.get_durations()
realms = dw.describe_realms()
aggression_units = dw.get_aggregation_units()

In [1]:
#Create dictionary of all possible callable dates
from datetime import date,timedelta

#yoinked from xdmod-data/xdmod_data/_validator.py
def __date_add_years(old_date, year_delta):
    # Make dates behave like Ext.JS, i.e., if a date is specified
    # with a day value that is too big, add days to the last valid
    # day in that month, e.g., 2023-02-31 becomes 2023-03-03.
    new_date_year = old_date.year + year_delta
    new_date_day = old_date.day
    days_above = 0
    keep_going = True
    while keep_going:
        try:
            new_date = date(new_date_year, old_date.month, new_date_day)
            keep_going = False
        except ValueError:
            new_date_day -= 1
            days_above += 1
    return new_date + timedelta(days=days_above)
def get_dates_from_duration():
    today = date.today()
    yesterday = today + timedelta(days=-1)
    last_week = today + timedelta(days=-7)
    last_month = today + timedelta(days=-30)
    last_quarter = today + timedelta(days=-90)
    this_month_start = date(today.year, today.month, 1)
    if today.month == 1:
        last_full_month_start_year = today.year - 1
        last_full_month_start_month = 12
    else:
        last_full_month_start_year = today.year
        last_full_month_start_month = today.month - 1
    last_full_month_start = date(
        last_full_month_start_year,
        last_full_month_start_month,
        1,
    )
    last_full_month_end = this_month_start + timedelta(days=-1)
    this_quarter_start = date(
        today.year,
        ((today.month - 1) // 3) * 3 + 1,
        1,
    )
    if today.month < 4:
        last_quarter_start_year = today.year - 1
    else:
        last_quarter_start_year = today.year
    last_quarter_start = date(
        last_quarter_start_year,
        (((today.month - 1) - ((today.month - 1) % 3) + 9) % 12) + 1,
        1,
    )
    last_quarter_end = this_quarter_start + timedelta(days=-1)
    this_year_start = date(today.year, 1, 1)
    this_year_end = date(today.year, 12, 31)
    previous_year_start = date(today.year - 1, 1, 1)
    previous_year_end = date(today.year - 1, 12, 31)
    durations_to_dates = {
        'Yesterday': (yesterday, yesterday),
        '7 day': (last_week, today),
        '30 day': (last_month, today),
        '90 day': (last_quarter, today),
        'Month to date': (this_month_start, today),
        'Previous month': (last_full_month_start, last_full_month_end),
        'Quarter to date': (this_quarter_start, today),
        'Previous quarter': (last_quarter_start, last_quarter_end),
        'Year to date': (this_year_start, today),
        'Previous year': (previous_year_start, previous_year_end),
        '1 year': (__date_add_years(today, -1), today),
        '2 year': (__date_add_years(today, -2), today),
        '3 year': (__date_add_years(today, -3), today),
        '5 year': (__date_add_years(today, -5), today),
        '10 year': (__date_add_years(today, -10), today),
        str(today.year): (this_year_start, this_year_end),
        str(__date_add_years(today, -1).year): (
            previous_year_start,
            previous_year_end,
        ),
    }
    for num_years in range(2, 7):
        durations_to_dates[str(__date_add_years(today, -num_years).year)] = (
            date(today.year - num_years, 1, 1),
            date(today.year - num_years, 12, 31),
        )
    return durations_to_dates
possible_dates = get_dates_from_duration()

NameError: name 'dw' is not defined

In [4]:
import csv
import base64

output_file = 'data_document.csv'
file_header = ["dur", "realm", "metrics_label", "dimension_label", "dataSetType", "aggUnit", "Description","URL"]  # TODO - Finalize format later

# Create a new CSV file to store data to, and add a header row, defined above
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(file_header)

    # Loop through all possible permutations of data querying (being duration, realm, metric, dimensions, filters, dataset_type, and aggregation)
    with dw:
        # Load the following data once to reduce calls
        
        # TODO - May need to find a better way for dw.describe_metrics, dimensions, and filter values to get a local copy
        # To reduce the number of calls and queries - These calls take an input which presently results in multiple calls being needed
        # Maybe make all necessary calls once, store them in a dictionary, and access the data from the dictionary with a key, instead of needing
        # to pass parameters to function calls

        for dur in durations:
            for realm in realms['label']:
                for metric_index, metric_row in dw.describe_metrics(realm).iterrows():
                    metrics_label = metric_row['label']
                    metrics_descriptions = metric_row['description']
                    for dim_index, dim_row in dw.describe_dimensions(realm).iterrows():
                        dimension_label = dim_row['label']
                        dimension_descriptions = dim_row['description']
                        for dataSetType in ["timeseries", "aggregate"]:  # These values are hard coded -> may cause error if new types are later introduced
                            for aggUnit in aggression_units:
#                                 #NOTE !!!! - have special case in filters, problem with if dimension is none - not sure what to do in that case
#                                 for filter_index, filter_row in dw.get_filter_values(realm, (dimension_label if dimension_label != "None" else "Resource")).iterrows():
#                                     filter_label = filter_row['label']

                                # dur - permutations of duration field
                                # realm - permutations of realms
                                # metrics_label - the value for the metric
                                # metrics_descriptions - metric description string
                                # dimension_label
                                # dimension_descriptions
                                # filter_label
                                # dataSetType
                                # aggUnit

                                # metric_index, dim_index (The ID fields of the tables), filter_index - might be used for URL generation???

                                # TODO - figure out what all gets added to the description field and where it comes from
                                Description = "Description: " + metrics_descriptions + "\n" + dimension_descriptions + "\n"
                                
                              # Create URL
                                duration = possible_dates[dur]
                                start_date =  duration[0].strftime("%Y-%m-%d")
                                end_date =  duration[1].strftime("%Y-%m-%d")
                    
                                config = """
                                {
                                  "featured": false,
                                  "trend_line": false,
                                  "x_axis": {},
                                  "y_axis": {},
                                  "legend": {},
                                  "defaultDatasetConfig": {},
                                  "swap_xy": false,
                                  "share_y_axis": false,
                                  "hide_tooltip": false,
                                  "show_remainder": false,
                                  "timeseries": true,
                                  "title": "untitled query 2",
                                  "legend_type": "bottom_center",
                                  "font_size": 3,
                                  "show_filters": true,
                                  "show_warnings": true,
                                  "data_series": {
                                    "data": [
                                      {
                                        "group_by": """+dimension_label+""",
                                        "color": "auto",
                                        "log_scale": false,
                                        "std_err": false,
                                        "value_labels": false,
                                        "display_type": "line",
                                        "combine_type": "side",
                                        "sort_type": "value_desc",
                                        "ignore_global": false,
                                        "long_legend": true,
                                        "x_axis": false,
                                        "has_std_err": true,
                                        "trend_line": false,
                                        "line_type": "Solid",
                                        "line_width": 2,
                                        "shadow": false,
                                        "filters": {
                                          "data": [],
                                          "total": 0
                                        },
                                        "z_index": 0,
                                        "visibility": null,
                                        "enabled": true,
                                        "metric": """+metric_index+""",
                                        "realm": """+realm+""",
                                        "category": """+realm+""",
                                        "id": 6558328419773369
                                      }
                                    ],
                                    "total": 1
                                  },
                                  "aggregation_unit": "Auto",
                                  "global_filters": {
                                    "data": [],
                                    "total": 0
                                  },
                                  "timeframe_label": """+dur+""",
                                  "start_date":"""+start_date+""",
                                  "end_date": """+end_date+""",
                                  "start": 0,
                                  "limit": 10
                                }
                                                                """
                                
                                # Encode to Base64
                                config_encoded = base64.b64encode(config.encode('utf-8'))      
                                URL = "https://xdmod-dev.ccr.xdmod.org/#main_tab_panel:metric_explorer?config="+config_encoded+"="
                                
                                writer.writerow([dur, realm, metrics_label, dimension_label, dataSetType, aggUnit, Description,URL])

#                                 # Get the data for a specific set of permutations
#                                 data = dw.get_data(
#                                     duration=dur,
#                                     realm=realm,
#                                     metric=metrics_label,
#                                     dimension=dimension_label,
#                                     filters={},
#                                     dataset_type=dataSetType,
#                                     aggregation_unit=aggUnit,
#                                 )




2024-05-29
2024-05-29
Yesterday


TypeError: can only concatenate str (not "datetime.date") to str